In [ ]:
# 파이썬으로 구현

import numpy as np

import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

In [ ]:
# 파이토치로 구현

import torch
import torch.nn as nn

input_size = 5
hidden_size = 8

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size, hidden_size, batch_first=True)

outputs, _status = cell(inputs)
print(outputs.shape)
print(_status.shape)

# Deep RNN
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape)
print(_status.shape)

# Bidirectional RNN
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional=True)
outputs, _status = cell(inputs)
print(outputs.shape)
print(_status.shape)

# LSTM
cell = nn.LSTM(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)

# GRU
cell = nn.GRU(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])
torch.Size([1, 10, 8])
torch.Size([2, 1, 8])
torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


In [5]:
# 문자 단위

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

index_to_char ={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(input_size, hidden_size, output_size)

outputs = net(X)
print(outputs.shape)

print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

print(Y.shape)
print(Y.view(-1).shape)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

문자 집합의 크기 : 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])
torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])
0 loss:  1.7013962268829346 prediction:  [[3 1 1 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  laap!
1 loss:  1.4020740985870361 prediction:  [[4 3 3 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pllp!
2 loss:  1.2104556560516357 prediction:  [[4 4 3 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplp!
3 loss:  1.0410974025726318 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.8741825222969055 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.703672289848

In [3]:
# 더 많은 데이터로 학습한 문자 단위

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

print(char_dic)

dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10 # 임의의 숫자 지정
learning_rate = 0.1

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1]

    x_data.append([char_dic[c] for c in x_str]) # 문자 -> 정수 인코딩
    y_data.append([char_dic[c] for c in y_str]) # 문자 -> 정수 인코딩

x_one_hot = [np.eye(dic_size)[x] for x in x_data] # 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))


class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size, dic_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape)

print(outputs.view(-1, dic_size).shape)
print(Y.view(-1).shape)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2) # 최종 예측값
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[c] for c in result])
        else: # 그 다음부터는 마지막 문자 하나씩만 추가
            predict_str += char_set[result[-1]]
            
    print(predict_str)

{'f': 0, 'g': 1, '.': 2, 's': 3, "'": 4, 'y': 5, ',': 6, 'n': 7, 'c': 8, 'i': 9, 'l': 10, 'm': 11, 'o': 12, 'd': 13, 'a': 14, 'b': 15, 'p': 16, 'k': 17, 't': 18, ' ': 19, 'w': 20, 'u': 21, 'e': 22, 'h': 23, 'r': 24}
문자 집합의 크기 : 25
훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
torch.Size([170, 10, 25])
torch.Size([1700, 25])
torch.Size([1700])
yeieeyieyyeieyiiyyeiiyi,lyeiieeyyeieyyyiynieyeyiyieyyyeeyyieyinsiiyneieeyyiyyyideyyeiye,yeyyeyyiiey,yyiyyyieeyyyiiyeiyyeyyyieyyesieyyiieyiiyyyiiyyieyyieyyiyy,isiiyyyyeyieiieyyi,yy
eddedededdededddededdddeeddddddddeddddddeddddddeddeddddedddedddddeddddeddddddddddddeddddddddeedddedddddddddeddddddeddddeedddedddedeedddeddeddddddddeedddddeddddeddndddedddddeeddedn
t  o   o    o ooe e o ooeo   o  e e  o eo eo   e oo e oe  oo oo ee  oeo   oo   o  e e oooe ooe   o eo o    o  o  oe e ooe  oo   e oe   o oe  e o  ooe oo  e  ooo     oo oo ooe ooo 
t to sto lt h dttle ttla lt dtldt d lh dtldtl    lh dt   tth dt tt t do   lsl ta dt dtlst 